In [1]:
import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [2]:
# Swish activation function
# x*sigmoid(x)
def swish(x):
    return x*K.sigmoid(x)

# Custom activation function 1
# mix between relu and positive part of swish mirrored across x=1
def e_swish_1(x):
    return K.maximum(0.0, x*(2-K.sigmoid(x)))

# Custom activation function 2
# positive part of swish mirrored across x=1
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

def leaky_relu(x):
    return K.maximum(0.3*x, x)

In [4]:
activations = [e_swish_1, e_swish_2, swish, "relu", leaky_relu, "elu", "selu"]
names = activations[:]
for i,a in enumerate(names):
    if not isinstance(a, str):
        names[i] = a.__name__
print(names)

['e_swish_1', 'e_swish_2', 'swish', 'relu', 'leaky_relu', 'elu', 'selu']


In [5]:
baseMapNum = 32
weight_decay = 1e-4

act, act_name = activations[4], names[4]
print(act_name)


model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

leaky_relu
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         

In [6]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [7]:
#training
batch_size = 128
epochs=25
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=3*epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep75'+act_name+'.h5')

Epoch 1/75
390/390 [==============================] - 161s - loss: 2.0072 - acc: 0.3993 - val_loss: 1.9533 - val_acc: 0.3916
Epoch 2/75
390/390 [==============================] - 155s - loss: 1.4303 - acc: 0.5461 - val_loss: 1.1115 - val_acc: 0.6429
Epoch 3/75
390/390 [==============================] - 155s - loss: 1.2548 - acc: 0.6038 - val_loss: 0.9825 - val_acc: 0.6969
Epoch 4/75
390/390 [==============================] - 155s - loss: 1.1622 - acc: 0.6330 - val_loss: 0.9253 - val_acc: 0.7073
Epoch 5/75
390/390 [==============================] - 155s - loss: 1.0688 - acc: 0.6569 - val_loss: 0.7886 - val_acc: 0.7568
Epoch 6/75
390/390 [==============================] - 155s - loss: 0.9983 - acc: 0.6744 - val_loss: 0.7906 - val_acc: 0.7482
Epoch 7/75
390/390 [==============================] - 155s - loss: 0.9524 - acc: 0.6914 - val_loss: 0.7698 - val_acc: 0.7612
Epoch 8/75
390/390 [==============================] - 153s - loss: 0.9213 - acc: 0.7024 - val_loss: 0.7386 - val_acc: 0.7696


390/390 [==============================] - 153s - loss: 0.6399 - acc: 0.8172 - val_loss: 0.5269 - val_acc: 0.8578
Epoch 67/75
390/390 [==============================] - 153s - loss: 0.6373 - acc: 0.8185 - val_loss: 0.5330 - val_acc: 0.8562
Epoch 68/75
390/390 [==============================] - 153s - loss: 0.6399 - acc: 0.8133 - val_loss: 0.5383 - val_acc: 0.8544
Epoch 69/75
390/390 [==============================] - 153s - loss: 0.6355 - acc: 0.8173 - val_loss: 0.5147 - val_acc: 0.8634
Epoch 70/75
390/390 [==============================] - 153s - loss: 0.6406 - acc: 0.8154 - val_loss: 0.5543 - val_acc: 0.8491
Epoch 71/75
390/390 [==============================] - 153s - loss: 0.6360 - acc: 0.8166 - val_loss: 0.5487 - val_acc: 0.8514
Epoch 72/75
390/390 [==============================] - 153s - loss: 0.6384 - acc: 0.8165 - val_loss: 0.5180 - val_acc: 0.8624
Epoch 73/75
390/390 [==============================] - 153s - loss: 0.6367 - acc: 0.8183 - val_loss: 0.5019 - val_acc: 0.8641
Epoc

In [8]:
opt_rms = keras.optimizers.rmsprop(lr=0.0005,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep100'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 156s - loss: 0.5872 - acc: 0.8344 - val_loss: 0.4638 - val_acc: 0.8768
Epoch 2/25
390/390 [==============================] - 154s - loss: 0.5744 - acc: 0.8385 - val_loss: 0.4621 - val_acc: 0.8792
Epoch 3/25
390/390 [==============================] - 153s - loss: 0.5694 - acc: 0.8387 - val_loss: 0.4844 - val_acc: 0.8691
Epoch 4/25
390/390 [==============================] - 153s - loss: 0.5638 - acc: 0.8406 - val_loss: 0.4516 - val_acc: 0.8776
Epoch 5/25
390/390 [==============================] - 153s - loss: 0.5559 - acc: 0.8412 - val_loss: 0.4626 - val_acc: 0.8766
Epoch 6/25
390/390 [==============================] - 153s - loss: 0.5592 - acc: 0.8416 - val_loss: 0.4683 - val_acc: 0.8743
Epoch 7/25
390/390 [==============================] - 153s - loss: 0.5543 - acc: 0.8423 - val_loss: 0.4549 - val_acc: 0.8784
Epoch 8/25
390/390 [==============================] - 153s - loss: 0.5470 - acc: 0.8453 - val_loss: 0.4492 - val_acc: 0.8772


In [9]:
opt_rms = keras.optimizers.rmsprop(lr=0.0003,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep125'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 154s - loss: 0.5055 - acc: 0.8538 - val_loss: 0.4282 - val_acc: 0.8828
Epoch 2/25
390/390 [==============================] - 153s - loss: 0.4970 - acc: 0.8580 - val_loss: 0.4237 - val_acc: 0.8864
Epoch 3/25
390/390 [==============================] - 153s - loss: 0.4934 - acc: 0.8605 - val_loss: 0.4079 - val_acc: 0.8882
Epoch 4/25
390/390 [==============================] - 153s - loss: 0.4947 - acc: 0.8582 - val_loss: 0.4129 - val_acc: 0.8874
Epoch 5/25
390/390 [==============================] - 153s - loss: 0.4869 - acc: 0.8610 - val_loss: 0.4182 - val_acc: 0.8878
Epoch 6/25
390/390 [==============================] - 153s - loss: 0.4903 - acc: 0.8598 - val_loss: 0.4158 - val_acc: 0.8869
Epoch 7/25
390/390 [==============================] - 153s - loss: 0.4880 - acc: 0.8591 - val_loss: 0.4082 - val_acc: 0.8872
Epoch 8/25
390/390 [==============================] - 153s - loss: 0.4858 - acc: 0.8604 - val_loss: 0.4117 - val_acc: 0.8848


In [10]:
#testing - no kaggle eval
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

 9984/10000 [============================>.] - ETA: 0s
Test result: 89.140 loss: 0.397
